<a href="https://colab.research.google.com/github/MyDearGreatTeacher/AI4high/blob/master/UsingColab_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 4.93 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [2]:
!ls

deep-learning-from-scratch  sample_data


In [0]:
!cd deep-learning-from-scratch

In [5]:
!ls deep-learning-from-scratch

ch01  ch03  ch05  ch07	common	 LICENSE.md
ch02  ch04  ch06  ch08	dataset  README.md


In [6]:
!ls deep-learning-from-scratch/ch01

hungry.py  img_show.py	man.py	simple_graph.py  sin_cos_graph.py  sin_graph.py


In [7]:
!python deep-learning-from-scratch/ch01/simple_graph.py

<Figure size 640x480 with 1 Axes>


In [8]:
!python deep-learning-from-scratch/ch01/hungry.py

I'm hungry!


In [9]:
!python deep-learning-from-scratch/ch04/gradient_method.py

<Figure size 640x480 with 1 Axes>
